<h1 align=center>Indo mais fundo – A mecânica do TensorFlow</h1>
<p align=center><img src=https://cdn.shortpixel.ai/spai/w_912+q_+ret_img+to_webp/https://iaexpert.academy/wp-content/uploads/2021/01/26.01.png width = 500></p>

Agora que temos alguma experiência prática com treinamento de uma Rede Neural (RN) do *TensorFlow* e aprendizado de máquina, é hora de mergulhar mais fundo na biblioteca do *TensorFlow* e explorar seu rico conjunto de recursos, o que nos permitirá implementar um aprendizado profundo mais avançado modelos nas próximas etapas.

Usaremos diferentes aspectos da API do *TensorFlow* para implementar as RNs. Em particular, usaremos novamente a API *Keras*, que fornece várias camadas de abstração para tornar a implementação de arquiteturas padrão muito conveniente. O *TensorFlow* também nos permite implementar camadas RNs personalizadas, o que é muito útil em projetos orientados à pesquisa que exigem mais personalização. Mais adiante, implementaremos essa camada personalizada.

Para ilustrar as diferentes formas de construção de modelos usando a API *Keras*, também consideraremos o problema clássico **exclusivo ou (XOR)**. Primeiramente, construiremos perceptrons multicamadas usando a classe `Sequential`. Em seguida, consideraremos outros métodos, como subclassificar `tf.keras.Model` para definir camadas personalizadas. Por fim, abordaremos o `tf.estimator`, uma API *TensorFlow* de alto nível que encapsula as etapas de aprendizado de máquina da entrada bruta à previsão.

### Como criar um gráfico no *TensorFlow* v1.x
Na versão anterior da API de baixo nível do *TensorFlow* (v1.x), esse gráfico precisava ser declarado explicitamente. As etapas individuais para criar, compilar e avaliar esse gráfico de computação no *TensorFlow* v1.x são as seguintes:
1. Instanciar um novo gráfico de computação vazio
2. Adicione nós (tensores e operações) ao gráfico de computação
3. Avalie (execute) o gráfico:
* Iniciar uma nova sessão
* Inicialize as variáveis ​​no gráfico
* Execute o gráfico de computação nesta sessão

Antes de darmos uma olhada na abordagem dinâmica no *TensorFlow* v2, vamos ver um exemplo simples que ilustra como criar um gráfico no *TensorFlow* v1.x para calcular $\small z = 2 \times (a-b) + c$. As variáveis $​​\small a$, $​​\small b$ e $​​\small c$ são escalares (números únicos) e definimos como constantes do *TensorFlow*. Um gráfico pode então ser criado chamando `tf.Graph()`. As variáveis, assim como os cálculos, representam os nós do gráfico, que definiremos da seguinte forma:

In [4]:
## TF v1.x style
import tensorflow as tf
g = tf.Graph()

with g.as_default():
    a = tf.constant(1, name='a')
    b = tf.constant(2, name='b')
    c = tf.constant(3, name='c')
    z = 2*(a-b) + c


Neste código, primeiro definimos o gráfico `g` via `g=tf.Graph()`. Em seguida, adicionamos nós ao gráfico, `g`, usando com `g.as_default()`. No entanto, observe que, se não criarmos um gráfico explicitamente, sempre haverá um gráfico padrão ao qual variáveis ​​e cálculos serão adicionados automaticamente.

No *TensorFlow* v1.x, uma sessão é um ambiente no qual as operações e os tensores de um gráfico podem ser executados. A classe `Session` foi removida do *TensorFlow* v2; No entanto, por enquanto, ele ainda está disponível por meio do submódulo `tf.compat` para permitir a compatibilidade com o *TensorFlow* v1.x. Um objeto de sessão pode ser criado chamando `tf.compat.v1.Session()`, que pode receber um gráfico existente (no caso, `g`) como argumento, como em `Session(graph=g)`.

Após lançar um gráfico em uma sessão do *TensorFlow*, podemos executar seus nós, ou seja, avaliar seus tensores ou executar seus operadores. Avaliar cada tensor individual envolve chamar seu método `eval()` dentro da sessão atual. Ao avaliar um tensor específico no gráfico, o *TensorFlow* precisa executar todos os nós anteriores no gráfico até atingir o nó de interesse fornecido. Caso haja uma ou mais variáveis *​​placeholder*, também precisamos fornecer valores para elas através do método `run` da sessão, como veremos mais adiante.

Depois de definir o gráfico estático no trecho de código anterior, podemos executar o gráfico em uma sessão do *TensorFlow* e avaliar o tensor, `z`, da seguinte forma:

In [5]:
## TF v1.x style

with tf.compat.v1.Session(graph=g) as sess:
    print('Result: z = ', sess.run(z))

Result: z =  1


### Como migrar um gráfico para o *TensorFlow* v2

Em seguida, vamos ver como esse código pode ser migrado para o *TensorFlow* v2. O *TensorFlow* v2 usa gráficos dinâmicos (em oposição aos estáticos) por padrão (isso também é chamado de execução antecipada no *TensorFlow*), o que nos permite avaliar uma operação em tempo real. Portanto, não precisamos criar explicitamente um gráfico e uma sessão, o que torna o fluxo de trabalho de desenvolvimento muito mais conveniente:

In [7]:
## TF v2 Style
a = tf.constant(1, name='a')
b = tf.constant(2, name='b')
c = tf.constant(3, name='c')

z = 2*(a - b) + c

tf.print('Result: z = ', z)



Result: z =  1


### Carregando dados de entrada em um modelo: estilo *TensorFlow* v1.x

Outra melhoria importante do *TensorFlow* v1.x para v2 diz respeito a como os dados podem ser carregados em nossos modelos. No *TensorFlow* v2, podemos alimentar dados diretamente na forma de variáveis *Python* ou arrays *NumPy*. No entanto, ao usar a API de baixo nível do *TensorFlow* v1.x, tivemos que criar variáveis de espaço reservado para fornecer dados de entrada a um modelo. Para o exemplo de gráfico de computação simples anterior, $\small z = 2 \times (a-b) + c$, vamos supor que `a`, `b` e `c` são os tensores de entrada de *Rank* 0. Podemos então definir três espaços reservados, que usaremos para "feed" dados para o modelo por meio de um dicionário *feed_dict*, da seguinte forma:

In [8]:
## TF-v1.x style
g = tf.Graph()
with g.as_default():
    a = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_a')
    b = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_b')
    c = tf.compat.v1.placeholder(shape=None, dtype=tf.int32, name='tf_c')
    z = 2*(a - b) + c
    
with tf.compat.v1.Session(graph=g) as sess:
    feed_dict = {a:1, b:2, c:3}
    print('Result: z =', sess.run(z, feed_dict=feed_dict))

Result: z = 1


### Carregando dados de entrada em um modelo: estilo *TensorFlow* v2

No *TensorFlow* v2, tudo isso pode ser feito simplesmente definindo uma função *Python* regular com `a`, `b` e `c` como seus argumentos de entrada, por exemplo:

In [10]:
## TF-v2 style
def compute_z(a, b, c):
    r1 = tf.subtract(a, b)
    r2 = tf.multiply(2, r1)
    z = tf.add(r2, c)
    return z

Agora, para realizar o cálculo, podemos simplesmente chamar essa função com objetos `Tensor` como argumentos de função. Observe que as funções do *TensorFlow*, como `add`, `subtract` e `multiply`, também nos permitem fornecer entradas de classificações mais altas na forma de um objeto *TensorFlow* `Tensor`, um array *NumPy* ou possivelmente outros objetos *Python*, como listas e tuplas. No exemplo de código a seguir, fornecemos entradas escalares (*Rank* 0), bem como entradas de *Rank* 1 e *Rank* 2, como listas:

In [11]:
tf.print('Scalar Inputs:', compute_z(1, 2, 3))
tf.print('Rank 1 Inputs:', compute_z([1], [2], [3]))
tf.print('Rank 2 Inputs:', compute_z([[1]], [[2]], [[3]]))

Scalar Inputs: 1
Rank 1 Inputs: [1]
Rank 2 Inputs: [[1]]
